GOAL: <br>
to find the stat sig features and then use these to train a model... <br>
need to find stat sig features inside the train / val / test split 

In [48]:
# PART OF THIS CODE IS FROM MICHAEL MURPHY - THANKS!
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
import glob, re
import seaborn as sns
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 100)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils.multiclass import type_of_target # used to check the Y labels are appropriate for classification
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.utils import shuffle
from scipy import interp
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from statsmodels.stats.multitest import multipletests
from sklearn.cross_decomposition import PLSRegression


def get_num_labels(ds):
    ds['labels'] = ds['labels']*1
    vals = ds['labels'].values
    try:
        vals = [item for sublist in vals for item in sublist]
    except:
        pass
    labels = set(vals)
    ds['num_labels'] = len(labels)
    ds['label_set'] = labels
    return ds

def check_pre_norm(ds):
    if ds['data_set'] in pre_norm_ds:
        ds['pre_norm'] = 'Yes'
    else:
        ds['pre_norm'] = 'No'
    return ds

def convert_nan_to_val(data, value=0):
    data[pd.isnull(data)] = value
    return data

In [55]:
#### Use this if DOING a fresh modeling fitting analysis
bn = True # use the percentile normalized data or no? 
log = False
stand_scaler = False
reanalysis = True
model = 'log_reg' #log_reg, rf or svm

if reanalysis:
    pre_norm_ds = [ 'plasmaall_author',
                    'urineall_author',
                    'm_oxylipin_chronic_hep_b',
                    'm_chronic_hep_b_POS',
                    'm_chronic_hep_b_NEG',
                    'm_CER_mass_spectrometry_v4',
                    'm_CER_mass_spectrometry_v4_3_CS',
                    'm_CER_mass_spectrometry_v4_0_NS',
                    'm_CER_mass_spectrometry_v4_2_FS',
                    'm_CER_mass_spectrometry_v4_1_COPD',
                    'm_EICO_mass_spectrometry_v4',
                    'm_EICO_mass_spectrometry_v4_3_CS',
                    'm_EICO_mass_spectrometry_v4_0_NS',
                    'm_EICO_mass_spectrometry_v4_2_FS',
                    'm_EICO_mass_spectrometry_v4_1_COPD',
                    'AN000580',
                    'AN000581',
                    'AN001503',
                    'ulsam_author']

    if bn:
        path = './bn_pickles/*.pkl'
    else:
        path = './pickles/*.pkl'

    datasets = OrderedDict()
    for fn in sorted(glob.glob(path)):
        data = pd.read_pickle(open(fn,'rb'))
        datasets[data[0]['study']] = data
    
else:
    #### Use this if NOT doing a fresh modeling fitting analysis
    pickle_file = './YES_bn_ds_models_and_sigfeat_NO_log_NO_standscal_NO_multi_mapped_labels.pkl'
    ### The non-batch corrected pickle for the dataset
    # pickle_file = './NO_bn_dataset_models_and_sigfeat_YES_log.pkl'
    datasets = pickle.load(open(pickle_file, 'rb'))
    

In [71]:
def train_model(X,y,ds,model):
    X,y = shuffle(X,y)
    if model == 'log_reg':
        if ds['num_labels'] != 2:
            clf = LogisticRegressionCV(scoring='accuracy', penalty='l1', solver='liblinear', tol=1e-4, intercept_scaling=1, max_iter=500, multi_class='ovr')
        else:
            clf = LogisticRegressionCV(scoring='roc_auc', penalty='l1', solver='liblinear', tol=1e-4, intercept_scaling=1, max_iter=500)
    elif model == 'rf':
        param_grid = {'n_estimators':[100,500,1000]}
        clf = GridSearchCV(RandomForestClassifier(n_estimators=1000, n_jobs=-1), param_grid, cv=3, n_jobs=-1)
    elif model == 'svm':
        param_grid = {'gamma': [1e-3, 0.01, 0.1, 1], 'C': [0.01, 0.1, 1, 10, 100]}
        clf = GridSearchCV(SVC(kernel='linear', probability=True), param_grid, cv=3, n_jobs=-1)
    elif model == 'plsda':
        param_grid = {'n_components': [2,3,5,10,50,100]}
        clf = GridSearchCV(PLSRegression(), param_grid, cv=3, n_jobs=-1)
    else:
        print('no valid classifier input, please try again with one of: log_reg, rf or svm')
        exit(0)
    cv = StratifiedKFold(n_splits=5, shuffle=True) # so this will probably give rather high - at the end you just get the last model...
    aucs = []
    num_stat = []
    for train, test in cv.split(X,y):
        x_train, y_train = X[train], y[train]
        x_test, y_test = X[test], y[test]
#         print('train, test pre: ',x_train.shape, x_test.shape)
        ###### i guess here you can find the sig features using the train data, and mask the x_train / then x_test 
        p = np.zeros(x_train.shape[1]) + np.nan
        for i in range(x_train.shape[1]):
            feat_data = []
            for j in set(y_train):
                try:
                    X_0 = x_train[y_train==j,i]
                    X_0 = X_0[~np.isnan(X_0)]
                    feat_data.append(X_0)
                except:
                    pass 
            if set(feat_data[0]) == set(feat_data[1]):
                p[i] = 1
                continue
            else:
                _, p[i] = mannwhitneyu(feat_data[0],feat_data[1], alternative='two-sided')                     
        try:
            _, p[~np.isnan(p)], _, _ = multipletests(p[~np.isnan(p)], alpha=0.05, method='fdr_bh')
        except:
            pass
        x_train = x_train[:,p<0.05]
        x_test = x_test[:,p<0.05]
        num_stat.append(x_train.shape[1])
        if x_train.shape[1] == 0:
            return 0.5, 0, ',', y_train.shape, y_test.shape, 0, np.asarray(num_stat).mean()
        if x_test.shape[1] == 0:
            return 0.5, 0, ',', y_train.shape, y_test.shape, 0, np.asarray(num_stat).mean()
#         print('train, test post: ', x_train.shape, x_test.shape)
        if stand_scaler:
            scaler = StandardScaler()
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.transform(x_test)
        clf.fit(x_train, y_train)
        y_pred = clf.predict_proba(x_test)
        fpr, tpr, _ = roc_curve(y_test, y_pred[:,1])
        auc_value = metrics.auc(fpr, tpr)
        aucs.append(auc_value) 
    auc = np.asarray(aucs)
    if ds['num_labels'] != 2:
        multi_aucs = auc
    else: multi_aucs = 0
    return auc.mean(), auc.std(), clf, y_train.shape, y_test.shape, multi_aucs, np.asarray(num_stat).mean()

def fit_model(X,y,ds,model):
    mean, std, clf, train_size, test_size, multi_aucs, num_stat =  train_model(X,y,ds,model)
    if mean == 1.0 or mean == 0.5:
        mean, std, clf, train_size, test_size, multi_aucs, num_stat = train_model(X,y,ds,model)
    return mean, std, train_size[0], test_size[0], clf, multi_aucs, num_stat

In [ ]:
# Used to fit models for all the datasets!
for k, v in datasets.items():  
    for ds in v:   
        print(k, ds['data_set'], ds['features'].shape)
        ds = get_num_labels(ds)
        ds = check_pre_norm(ds)                
        y = ds['labels'].values.copy().ravel().astype(int)    
        X = ds['features'].values.copy()
        X = convert_nan_to_val(X, value=0)
        X[np.isinf(X)] = 0
        X[X<0] = 0
        if log and ds['pre_norm'] == 'No':
            print('using log')
            X[X<1] = 1
            X = np.log2(X)
        aucs = []
        avg_stat_sig = []
        for i in range(30):
            auc, std,train_size,test_size,clf,multi_auc, num_stat = fit_model(X,y,ds,model)
            aucs.append(auc)
            avg_stat_sig.append(num_stat)
        ds['avg_stat_sig'] = np.asarray(avg_stat_sig).mean()
        aucs = np.asarray(aucs)
        ds['auc'] = aucs.mean()
        ds['auc_std'] = aucs.std()
        ds['train_size'], ds['test_size'], ds['clf'], ds['multi_aucs'] = train_size, test_size, clf, multi_auc
#         ds['auc'], ds['auc_std'], ds['train_size'], ds['test_size'], ds['clf'], ds['multi_aucs'] = fit_model(X, y, ds, model=model)     
        print(ds['auc'],ds['auc_std'])

Feng plasmaall_author (102, 109)


In [ ]:
disease_type = {
    'acute myocardial infarction': 'cardiovascular',
    'cardiovascular': 'cardiovascular',
    'coronary heart disease': 'cardiovascular',
    'hepatocellular carcinoma': 'cancer',
    'Hepatocellular carcinoma': 'cancer',
    'Hepatocellular Carcinoma': 'cancer',
    'hepatitis b': 'infectious',
    'Malaria': 'infectious',
    'Malaria (P. vivax)':'infectious',
    'non-malaria febrile illness':'infectious',
    'scleroderma PAH': 'autoimmune',
    'psoriasis':'autoimmune',
    'pneumonia': 'infectious',
    'Pneumonia - Community acquired': 'infectious',
    'copd': 'respiratory',
    'COPD': 'respiratory',
    'chronic hepatitis B' : 'infectious',
    'typhoid': 'infectious',
    'typhoid carriage':'infectious',
    'lyme': 'infectious',
    'common cold - longitudinal':'infectious',
    'Lyme disease': 'infectious',
    'Alzheimers': 'neurological',
    "Alzheimer's": 'neurological',
    'colorectal cancer': 'cancer',
    'Colorectal Cancer': 'cancer',
    'depression': 'neurological',
    'Depression':'neurological',
    'Breast Cancer': 'cancer',
    'Breast cancer':'cancer',
    'Lung cancer': 'cancer',
    'lung cancer': 'cancer',
    'Lung Cancer': 'cancer',
    'lung cancer - adenocarcinoma': 'cancer',
    'lung cancer - non-small-cell lung cancer (adenocarcinoma, etc)': 'cancer',
    'Stability of dried blood samples - diabetic men' : 'metabolic',
    'Obesity - Non-diabetic and T2 diabetic': 'metabolic',
    't2 diabetes': 'metabolic',
    't1 diabetes': 'metabolic',
    'Diabetes - Type I': 'metabolic',
    'Diabetes - healthy v. T2 v. prediabetic': 'metabolic',
    'Polycystic Ovarian Syndrome': 'metabolic',
    'minimal change disease, focal segmental sclerosis': 'glomerular',
    'interstitial cystitis/painful bladder syndrome': 'other',
    'prepubertal children with obesity': 'other', #MAYBE CHANGE THIS ONE?
    'chronic fatigue syndrome': 'other',
    'Chronic fatigue': 'other',
    'polycystic ovarian syndrome': 'other',
    'scleroderma': 'other',
    'Pregnancy': 'other',
    'smoker v. nonsmoker':'other',
    'Interperson variation':'other',
    'short-term and long-term metabolic changes after bariatric surgery':'other',
    'high intensity exercise metabolomics':'other',
    'Age related metabolomics': 'other',
    'Urine sample storage': 'other',
    'urine metabolome': 'other',
    'Single human time study': 'other'
    }

def make_summary(u,l,i,k,j=0, replace=False):
    if replace:
        auc = u['multi_aucs'].mean(0)[j]
        auc_std = u['multi_aucs'].std(0)[j]
        analysis = u['data_set']+'_'+str(j)
        label = str(l)+str(i)+str(j)
        if model == 'log_reg':
            model_coef = np.count_nonzero(u['clf'].coef_[j])
    else:
        auc = u['auc']
        auc_std = u['auc_std']
        analysis = u['data_set']
#         print(str(l), str(i))
        label = str(l)+str(i)
#         if model == 'log_reg':
#             print(u['clf'])
#             model_coef = np.count_nonzero(u['clf'].coef_)
    s = {'disease': u['disease'], 
        'number_labels': 2,
        'auc':auc,
        'auc_std': auc_std,
        'stat_sig_feat':u['avg_stat_sig'],
        'label': label,
        'analysis': analysis,
        'disease_type': disease_type[u['disease']],
        'study': k}
#     if model == 'log_reg':
#         s['model_nonzero_coef'] = model_coef
    return s
    
from string import ascii_letters
summary = []
for l,k in zip(ascii_letters, datasets):
    for i, u in enumerate(datasets[k]):
        if (k == 'ST000062' and u['data_set'] == 'XCMS-Report-annotated-SingleClass-GCTOF.'):
            u['data_set'] = 'XCMS-Report-annotated-SingleClass-GCTOF.plasma'
        if u['num_labels'] == 2:
            control = u['labels']==0
            case = u['labels']==1
            try:
                summed_control = int(control.sum())
                summed_case = int(case.sum())
            except:
                pass
            summary.append(make_summary(u,l,i,k))
        else:
            for j in range(u['num_labels']):
                summary.append(make_summary(u,l,i,k,j=j,replace=True))
                #### Issues with multi class: the case and control just use the last datasets values, not real                      
summary = pd.DataFrame(summary)
# summary = summary.set_index('study')
# summary['disease_type'] = summary['disease_type'].astype('category')
summary

In [ ]:
# save the df as a csv:
summary.to_csv('./onlysigfeat_withnumbers_30avg_auc_{}_sigfeat_summary_YES_bn_NO_log_NO_standscal_YES_ovo.csv'.format(model))
# save the df as a pickle
summary.to_pickle('./onlysigfeat_withnumbers_30avg_auc_{}_sigfeat_df_YES_bn_NO_log_NO_standscal_YES_ovo.pkl'.format(model))
# save dataset object:
pickle.dump(datasets, open('./onlysigfeat_withnumbers_30avg_{}_YES_bn_ds_models_and_sigfeat_NO_log_NO_standscal_YES_ovo.pkl'.format(model), 'wb'))

In [ ]:
# map the extra metadata onto this data (column, mode, sample type)
# metadata = pd.read_csv('./ms_instrument_column_polarity_dataset_names.csv', sep='\t')
metadata = pd.read_csv('./ms_instrument_column_polarity_dataset_names.csv', sep='\t').set_index('Accession')
summary_w_metadata = summary.merge(metadata, on='analysis')
summary_w_metadata = summary_w_metadata.replace(np.nan,'unknown')
summary_w_metadata.to_csv('./1onlysigfeat_withnumbers_30avg_{}_auc_sigfeat_summary_YES_bn_NO_log_NO_standscal_YES_ovo_YES_meta.csv'.format(model))